## {{cookiecutter.project_name}}

{{cookiecutter.description}}

### Data Sources
- file1 : Description of where this file came from

### Changes
- {% now 'utc', '%m-%d-%Y' %} : Started project

In [ ]:
import pandas as pd
import numpy as np
import glob
import datetime as dt
import pickle
import os
from pandas import ExcelWriter
import re
from zipfile import ZipFile
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

### Functions

In [ ]:
def categorize_columns(df, numunique= 205):
    '''clean names of columns and caegorize object columns if unique is less than , numunique
       downcast for int and floats'''
    # https://stackoverflow.com/questions/30763351/removing-space-in-dataframe-python
    df.columns = [x.strip() for x in df.columns]
    df.rename(columns = lambda x: x.replace(" ","_"), inplace= True)
    df.rename(columns = lambda x: x.replace(".","_"), inplace= True)
    df.rename(columns = lambda x: x.replace("&","and"), inplace= True)
    df.rename(columns = lambda x: x.replace("(",""), inplace= True)
    df.rename(columns = lambda x: x.replace(")",""), inplace= True)
    df.rename(columns = lambda x: x.replace("/",""), inplace= True)
    df.rename(columns = lambda x: x.replace("-","_"), inplace= True)
    df.rename(columns = lambda x: x.replace("+","_plus_"), inplace= True)
    for y in df.columns:
        #categorize columns
        if df[y].dtype == np.object:
            if len(df[y].unique()) <= numunique:
                print('converted ' + y + ' ' +str(df[y].dtype) + ' records=' + str(len(df[y].unique())))
                df[y] = df[y].astype('category')
        elif(df[y].dtype == np.float64 or df[y].dtype == np.int64):
            df[y] = pd.to_numeric(df[y], downcast='unsigned')
            print('DOWNCAST ' +  y + ' ' +str(df[y].dtype))
    return df

def add_business_days(from_date, ndays):
    '''Consider weekends when add days to a date'''
    business_days_to_add = abs(ndays)
    current_date = from_date
    sign = ndays/abs(ndays)
    while business_days_to_add > 0:
        current_date += datetime.timedelta(sign * 1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date

### File Locations

In [ ]:
fileticketsname = input('Tickets file name: ')
fileappsname = input('Apps file name: ')
print(os.getcwd())
originalpath = (os.getcwd())
print(originalpath)

os.chdir(originalpath)
#os.chdir('..')
path = os.getcwd()
print(path)

#filename = os.path.join(path, 'data','Power - Tickets - 2018 - v2.1-2.xlsx')
filename = os.path.join(path, 'data','raw',fileticketsname)
fileapps = os.path.join(path, 'data', 'raw', fileappsname)

#outputfiles
today = datetime.datetime.today()
directory_name = '{{cookiecutter.directory_name}}'
fileoriginaltickets = os.path.join(path, 'data','processed',
                                   directory_name + '_tickets{:%m%d%y}.pkl').format(today)
fileoriginalapps = os.path.join(path, 'data','processed',
                                directory_name + '_apps{:%m%d%y}.pkl').format(today)

### Read files from Excel or CSV

In [ ]:
# if it is a CSV file ->tickets_file = pd.read_csv(filename)
tickets_file = pd.ExcelFile(filename, 
                            converters= {'opened_at': pd.to_datetime, 'closed_at': pd.to_datetime})
print(tickets_file.sheet_names)
#get list of countries ina dataframe

#read applications file
apps_file = pd.ExcelFile(fileapps)
print(apps_file.sheet_names)


In [ ]:
sheetname= input('Sheet to load for tickets: ')
tickets_final = tickets_file.parse(sheetname, skiprows=0)
print(len(tickets_final))

sheetname= input('Sheet to load foro apps: ')
apps_final = apps_file.parse(sheetname, skiprows=0)
print(len(apps_final))


### Column Cleanup

- Remove all leading and trailing spaces
- Get columns names and determine columns to rename df.columns
- Rename the columns for consistency.


In [ ]:
#clean name columns, categorize columns, downcast columns
tickets_final= categorize_columns(tickets_final)
apps_final= categorize_columns(apps_final)

In [ ]:
cols_to_rename = {'col1': 'New_Name'}
tickets_final.rename(columns=cols_to_rename, inplace=True)

In [ ]:
tickets_final.columns

### Clean Up Data Types

In [ ]:
df.dtypes

### Data Manipulation

### Save output file into processed directory
Save a file in the processed directory that is cleaned properly. It will be read in and used later for further analysis.

In [ ]:
#save file a pkl for fast access
tickets_final.to_pickle(fileoriginaltickets, 'gzip')
print('original tickets file saved as {}'.format(fileoriginaltickets))

#save file a pkl for fast access
apps_final.to_pickle(fileoriginalapps, 'gzip')
print('original apps file saved as {}'.format(fileoriginalapps))